## Strip Optimization

Importing the libraries needed for running the programme

In [1]:
# Loading libraries
import pandas as pd
import numpy as np
import math 
from decimal import ROUND_DOWN, ROUND_UP
# import data 

### Data for algorithm

In [2]:
# Initial variables for the calculation and validation
Frequency = 47 # Hz
Temperature_rise_goal = 30 # degree Celcius
Output_power = 250 # watts
Efficiency = 95 # %
Input_voltage = 115 # volts
Ouptut_voltage = 115 # volts
Regulation = 5 # in a scale of 100
Bobbin_thickness = 1.5 
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 250
insulation_thickness = 0.2 #mm

Some constants for calculation

In [3]:
pi = np.pi #pi
a = 1.68 # coefficients for core loss
b = 1.86 # coefficients for core loss 
Rate_of_Cu = 950 # Rs / Kg
Rate_of_Fe = 250 # Rs / Kg
Resistivity_Cu =  1.68 * 10**-6 # ohm cm

In [4]:
Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
print(Apparent_power)

513.1578947368421


In [5]:
def area_product(K_f, K_u, B_ac, J, Frequency, apparent_power):
    """Input Arguments:
        Output power watts
        Efficiency in scale of 100%
        K_f
        K_u
        B_ac -> Operational flux density
        J
        Frequency -> in Hz
        Apparent power in watts
    """
    Area_product = (apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
    return Area_product

Area_product = area_product(K_f, K_u, B_ac, J, Frequency, Apparent_power)
print(Area_product)

153.6916255561272


## Data Collection

* Lamination data
* SWG data

In [6]:
lamination_data = pd.read_csv('../DATA/EI-Laminations.csv')
swg_data = pd.read_csv('../DATA/EMD - Sheet1.csv') # select the swg the data 
# lamination_data = 

In [7]:
import data

print(data.swg_data.iloc[:2])

    SWG     mm CD Min mm CD Max mm  Normal Conductor Area mm²  \
0  10.0  3.240     3.218     3.284                    8.24479   
1  11.0  2.946     2.917     2.975                    6.81640   

   Conductor Weight for 1000m/Kg  CR Normal   CR Max     CR Min  FC Min mm  \
0                         73.296   0.002030  0.002090  0.001980        NaN   
1                         60.598   0.002529  0.002579  0.002479      0.055   

   FC Max mm  Medium Covering Min  Medium Covering Max  
0        NaN                0.086                3.380  
1      3.055                0.081                3.087  


In [8]:
def findSWG(Bare_area):
    '''
        Bare area in sqmm
        return variables: 
            required swg dataframe, 
            diameter of insulated wire, 
            bare area of selected swg
    '''
    swg_data = pd.read_csv('../DATA/EMD - Sheet1.csv') # select the swg the data 
    higer_data = swg_data[Bare_area < swg_data['Normal Conductor Area mm²']]
    required_swg_result = higer_data.iloc[(higer_data['Normal Conductor Area mm²'] - Bare_area).abs().argsort()[:1]]
    diameter_of_insulated_wire = required_swg_result['Medium Covering Max']
    A_wp = required_swg_result['Normal Conductor Area mm²'].max() / 100 # cm^2
    return required_swg_result, diameter_of_insulated_wire, A_wp 

def bare_area(input_current, current_density):
    '''
        arguments: 
            input current [amps], 
            current density [amp/]
        return variables: Bare area in sqmm
    '''
    a_wp = input_current / current_density
    return a_wp* 100 

Input_current = Apparent_power / (Input_voltage * 0.01*  Efficiency)
A_wp_in_sqmm = bare_area(Input_current, J)
required_swg_primary, diameter_of_primary_wire_with_insulation, A_wp = findSWG(A_wp_in_sqmm)
# results


In [9]:
print('Input current ' + str(Input_current) + ' [amps]')
print('Calculated Priamry Bare Area: ' + str(A_wp_in_sqmm)+ ' mm²')
print('Final Priamry Bare Area after selecting swg: ' + str(A_wp*100)+ ' mm²')
print('SWG: ' + str(required_swg_primary['SWG'].to_string(index=False)))
print('Primary wire Diameter with enamel: ' + str(diameter_of_primary_wire_with_insulation.max()) + ' mm')

Input current 4.69709743466217 [amps]
Calculated Priamry Bare Area: 1.8788389738648679 mm²
Final Priamry Bare Area after selecting swg: 2.0765 mm²
SWG: 16.0
Primary wire Diameter with enamel: 1.737 mm


## Algorithm

Strip cost = function(stack, tongue, ww, wl)

* Stack in mm
* Tongue in mm
* ww \[Widning width\] in mm
* wl \[Winding length\] in mm